In [22]:
import numpy as np
import json
from collections import defaultdict
import pandas as pd
import glob

In [23]:
def join_weights(rows):
    W = defaultdict(lambda:0, json.loads(rows.at(0)['weights']))
    for row in rows[1:]:
        r = json.loads(row['weights'])
        for k,v in r.items():
            W[k] += v
    return json.dumps(W)
    

def _step_7(file):
    chunk = pd.read_csv(file, sep='\t', header=None, names=['author_id', 'weights']) # _complete_short_
    out = open('data/AuthorsFOS_split/authors_weights_year_2020_weights_process_%s' % file.split('_')[-1], 'w')
    current = chunk.iloc[0,0]
    weights = defaultdict(lambda:0)
    for idx, row in chunk.iterrows():
        if current == row['author_id']:
            w = json.loads(row['weights'])
            for k, v in w.items():
                weights[k] += v
        else:
            out.write("%s\t%s\n" % (current, json.dumps(weights)))
            current = row['author_id']
            weights = defaultdict(lambda:0, json.loads(row['weights']))
    
    out.write("%s\t%s\n" % (current, json.dumps(weights)))
    out.close()
    del chunk
    
    
def step_7():
    files = glob.glob('data/AuthorsFOS_split/authors_weights_year_2020_split_sorted_*.csv')
    print(files[:3])
    from tqdm.contrib.concurrent import process_map
    process_map(_step_7, files, max_workers=16)

In [ ]:
%%time
step_7()

data/authors_weights_year_2020_sorted.csv
